<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">Sentiment Analysis for "AAPL" using twitter data - Yearly data</h2>
<p>This notebook will examine the relationship between twitter sentiment and stock price.</p>
<h2 style="color: darkblue;">Part 1 of the notebook focusses on sentiment analysis</h2>
<p>I have used NLTK's built-in, pretrained sentiment analyzer VADER (Valence Aware Dictionary and sEntiment Reasoner).
The pre-trained sentiment analyzer returns the 'compound' score for each tweet. This score ranges from -1 (most negative) to +1 ( most positive).</p>
<h2 style="color: darkblue;">Part 2 of the notebook establishes the relationship between twitter sentiment & stock price</h2>
<p>I have downloaded the stock data for Apple from yfinance.</p>
</div>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as tkr
import seaborn as sns
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# import tweets
tweets=pd.read_csv('/kaggle/input/tweets-about-the-top-companies-from-2015-to-2020/Tweet.csv')
company_tweet=pd.read_csv('/kaggle/input/tweets-about-the-top-companies-from-2015-to-2020/Company_Tweet.csv')

tweets=tweets.merge(company_tweet,how='left',on='tweet_id')
# format dates
tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
tweets.date=pd.to_datetime( tweets.date,errors='coerce')
tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">Table/Dataframe for all tweets related to apple in 2018</h2>
<p>View the tweet data for apple below. The data shows twitter handle/name along with the body. The body text is used to estimate the sentiment score.</p>
</div>


In [ ]:
tweets.head()

In [ ]:
sia = SentimentIntensityAnalyzer()

def get_sentiment(tweets,ticker='AAPL',start='2018-01-01',end='2018-12-31'):
    #sbuset
    df=tweets.loc[((tweets.ticker_symbol==ticker)&(tweets.date>=start)&(tweets.date<=end))]
    # applt the SentimentIntensityAnalyzer
    df.loc[:,('score')]=df.loc[:,'body'].apply(lambda x: sia.polarity_scores(x)['compound'])
    # create label
    #bins= pd.interval_range(start=-1, freq=3, end=1)
    df.loc[:,('label')]=pd.cut(np.array(df.loc[:,'score']),bins=[-1, -0.66, 0.32, 1],right=True ,labels=["bad", "neutral", "good"])
    
    df=df.loc[:,["date","score","label","tweet_id","body"]]
    return df

print('apple misses earnings, analyst suggest downgrade , sell now ')
sia.polarity_scores('apple misses earnings, analyst suggest downgrade , sell now ')

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">On augmenting the library with stock market jargons / lingo, the sentiment score changes & is a better representation of the actual sentiment</h2>
<p>For the same comments/tweets as in the previous cell, the score becomes more negative (skewed towards negative) after augmenting the library. Notice the results from the below cell.</p>
</div>

In [ ]:
# augment vocab

positive_words='high profit Growth Potential Opportunity Bullish Strong Valuable Success Promising Profitable Win Winner Outstanding Record Earnings Breakthrough buy bull long support undervalued underpriced cheap upward rising trend moon rocket hold breakout call beat support buying holding'
negative_words='resistance squeeze cover seller Risk Loss Decline Bearish Weak Declining Uncertain Troubling Downturn Struggle Unstable Volatile Slump Disaster Plunge sell bear bubble bearish short overvalued overbought overpriced expensive downward falling sold sell low put miss'

dictOfpos = { i : 4 for i in positive_words.split(" ") }
dictOfneg = { i : -4 for i in negative_words.split(" ")  }
Financial_Lexicon = {**dictOfpos, **dictOfneg}

sia.lexicon.update(Financial_Lexicon)


print('apple misses earnings, analyst suggest downgrade , sell now ')
sia.polarity_scores('apple misses earnings, analyst suggest downgrade , sell now ')

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">NOTE</h2>
<p>Below table highlights the sentiment score and the label as estimated using the nltk pre-trained library VADER..</p>
</div>



In [ ]:
start='2018-01-01'
end='2018-12-31'
ticker='AAPL'
tw=get_sentiment(tweets,ticker,start,end) # get tweets
tw.head()

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">Exhibit</h2>
<p>Distribution of positive, negative & neutral sentiment by each day.</p>
</div>

In [ ]:
import matplotlib.patches as mpatches

# Convert the 'date' column to datetime objects
tw['date'] = pd.to_datetime(tw['date'])

# Group the data by date and 'label' and count the number of tweets
sentiment_counts = tw.groupby(['date', 'label']).size().unstack(fill_value=0)

# Create an area chart
fig, ax = plt.subplots(figsize=(12, 6))

# Set colors for 'good,' 'neutral,' and 'bad'
colors = {'good': 'green', 'neutral': 'grey', 'bad': 'red'}

area = sentiment_counts.plot.area(stacked=True, ax=ax, color=[colors[c] for c in sentiment_counts.columns])

# Customize the legend
handles = [mpatches.Patch(color=colors[label], label=label.capitalize()) for label in sentiment_counts.columns]
ax.legend(handles=handles, title='Sentiment', loc="upper left")

# Customize the chart
plt.title('Sentiment Distribution by Date (2018)')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')

plt.show()


<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">NOTE</h2>
<p>The sentiment score is created for each tweet and hence there are multiple sentiment scores for each day. The below code averages out the sentiment score for each day. The average score is then plotted as a line graph. It is noted that the average sentiment score is always positive due to the fact that the overall sentiment score for each day throughout the year is positive (or in other words, there are more positive or neutral tweets that negative tweets). This can also be observed in the above chart which highlights the frequency of positive, negative & neutral tweets.</p>
</div>




In [ ]:
# Convert the 'date' column to datetime objects
tw['date'] = pd.to_datetime(tw['date'])

# Group the data by date and calculate the average sentiment score for each day
daily_sentiment = tw.groupby(tw['date'].dt.date)['score'].mean()

daily_sentiment_df = pd.DataFrame({'Date': daily_sentiment.index, 'Average Score': daily_sentiment.values})

# Print the resulting DataFrame
daily_sentiment_df.head()

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">Exhibit</h2>
<p>Average sentiment score for tweets related to Apple for each day of the year 2018.</p>
</div>

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the average sentiment scores
ax.plot(daily_sentiment.index, daily_sentiment, marker='o', linestyle='-', color='b')

# Set labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Average Sentiment Score')
plt.title('Average Sentiment Score for Each Day for the year 2018')

# Format x-axis date labels
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
!pip install yfinance

<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">The apple stock data is obtained from yfinance.</h2>
<p>1. The first plot highlights the stock price since beginning. 
2. The second plot higlights the stock price change over 2018 (full year).</p>
</div>

In [ ]:
import yfinance as yf
aapl = yf.Ticker("AAPL")
hist = aapl.history(period="max")
hist["Open"].plot(figsize=(15, 5), title="AAPL Stock Price")
plt.show()

In [ ]:
# Define the stock symbol and date range
stock_symbol = "AAPL"
start_date = "2018-01-01"
end_date = "2018-12-31"

# Create a Ticker object for the stock
stock = yf.Ticker(stock_symbol)

# Fetch historical data for the specified date range
hist = stock.history(period="1d", start=start_date, end=end_date)

# Plot the opening prices for the specified date range
hist["Close"].plot(figsize=(15, 5), title=f"{stock_symbol} Stock Price (Jan 1, 2018 - Dec 31, 2018)")
plt.show()

In [ ]:
# Define the start and end dates
start_date = '2018-01-01'
end_date = '2018-12-31'

# Create a date range covering the full date range (including weekends and holidays)
date_range = pd.date_range(start=start_date, end=end_date)

# Create a new DataFrame with the full date range
full_hist = pd.DataFrame(index=date_range)

# Ensure that the timezone information for both DataFrames matches
full_hist.index = full_hist.index.tz_localize(hist.index.tz)

# Merge or combine the new DataFrame with the existing 'hist' DataFrame
full_hist = full_hist.combine_first(hist)

# Now, 'full_hist' contains all dates within the specified date range
# with weekends and holidays included, and missing values are filled with NaN.

# If you want to fill NaN values with 0, you can do:
full_hist = full_hist.fillna(0)

# Print or use the updated DataFrame 'full_hist'
full_hist.head()


<div style="background-color: lightblue; padding: 10px; border: 1px solid blue;">
<h2 style="color: darkblue;">Exhibit</h2>
<p>Plotting the sentiment and the stock price in one chart so as to spot any trend/patterns between the two, through manual eye-balling
.</p>
</div>

In [ ]:
import matplotlib.pyplot as plt

# Extract data from dataframes
dates = full_hist.index
stock_prices = full_hist['Close']
sentiment_scores = daily_sentiment_df['Average Score']

# Create a figure and axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot stock prices as a bar chart on the first axis (left y-axis)
ax1.bar(dates, stock_prices, color='lightblue', label='Stock Price')
ax1.set_xlabel('Date')
ax1.set_ylabel('Stock Price', color='b')

# Create a second axis (right y-axis) for sentiment scores
ax2 = ax1.twinx()
ax2.plot(dates, sentiment_scores, color='darkred', label='Sentiment Score')
ax2.set_ylabel('Sentiment Score', color='r')

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

# Set a title
plt.title('Apple Stock Price (Bar Chart) and Average Twitter Sentiment Score (Line Chart)')

# Show the plot
plt.show()